In [239]:
import requests
from bs4 import BeautifulSoup
import re

def getlinks():
    f = open("naturelinks.txt","w")
    s = ""
    url = "https://www.nature.com/search?article_type=protocols%2Cresearch%2Creviews&subject=medical-research&fbclid=IwAR2bryCX-MVeF8y8qbtnFmlyjj6rdDzVwX7KV4ebEDB4W0xWj30boPX5Y2k"
    for page in range(1,427):
        r = requests.get(url + "&page=" + str(page))
        soup = BeautifulSoup(r.text, 'html.parser')
        results = soup.find_all('div', attrs={"class":"cleared"})
        for results in soup.find_all('h2'):
            for link in results.find_all('a'):
                s = s + "https://www.nature.com" + link.get('href') + "\n"
    f.write(s)

In [242]:
import threading,queue
q = queue.Queue()

def getdata(url):
    r = requests.get(url)
    content = ""
    soup = BeautifulSoup(r.text, 'html.parser')
    res =  soup.find_all('section')
    needed = ["Abstract","Introduction","Results","Discussion","Materials and Methods","Key points"]
    data = ""
    for sec in res:
        heading = sec.find('h2')
        if heading is not None:
            headingText = heading.get_text()
            if headingText in needed:
                paras = sec.find_all('p')
                for para in paras:
                    if para is not None:
                        data += para.get_text()
    data = (data.strip() + "\n")
    return data

def pushtoqueue(file):
    for line in file:
        q.put(line)

def threadfunc(file):
    fileD = open(file,"a")
    data = ""
    while not q.empty():
        try:
            print(url)
            url = q.get()
            data = data + (getdata(url)+"\n"+"\n")
        except:
            True
    print("done")
    #fileD.write(data)

#print(getdata("https://www.nature.com/articles/s41386-019-0377-9"))

In [243]:
q = queue.Queue()
f = open("templinks.txt","r")
dataFile = "naturedata.txt"
pushtoqueue(f)
numthreads = 1
threads = []
print(q.qsize())


for i in range(numthreads):
        threads.append(threading.Thread(target=threadfunc,args=(dataFile,)))
        threads[i].start()
     

2
